In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade pandas

In [ ]:
import pickle
import numpy as np
import pandas as pd
#pd.set_option('display.max_row', 10000)
#pd.set_option('display.max_columns', 100)
#pd.set_option('display.max_colwidth', 5000)

In [ ]:
# Link to save deduplication file
link_d_dedup = '/content/drive/MyDrive/NIA_NLP/DATA/DAUM/SOCIETY/10200_2102_fin.bin'

In [ ]:
daum_link1 = '/content/drive/MyDrive/NIA_NLP/DATA/DAUM/SOCIETY/10200_210512.bin'
daum_link2 = '/content/drive/MyDrive/NIA_NLP/DATA/DAUM/SOCIETY/10200_210524.bin'
daum_link3 = '/content/drive/MyDrive/NIA_NLP/DATA/DAUM/SOCIETY/10200_210605.bin'
daum_link4 = '/content/drive/MyDrive/NIA_NLP/DATA/DAUM/SOCIETY/10200_210616.bin'

In [ ]:
with open(daum_link1, 'rb') as f:
  df_daum_link1 = pickle.load(f)

In [ ]:
with open(daum_link2, 'rb') as f:
  df_daum_link2 = pickle.load(f)

In [ ]:
with open(daum_link3, 'rb') as f:
  df_daum_link3 = pickle.load(f)

In [ ]:
with open(daum_link4, 'rb') as f:
  df_daum_link4 = pickle.load(f)

In [ ]:
daum_frames = [df_daum_link1, df_daum_link2, df_daum_link3, df_daum_link4]

In [ ]:
df_daum = pd.concat(daum_frames, ignore_index=True)

In [ ]:
df_daum

In [ ]:
# remove duplicate links
df_dedup = df_daum.drop_duplicates(['Link'], keep = 'last')
df_dedup

In [ ]:
# remove spaces in the press name
df_dedup["Press"] = df_dedup["Press"].str.replace(" ", "")

In [ ]:
len(df_dedup)

In [ ]:
print(df_dedup.dtypes, "\n")

In [ ]:
# check duplicate article rows
df_dedup[df_dedup.duplicated(['Date', 'Title', 'Content', 'Press'], keep=False)]

In [ ]:
import itertools

# combine duplicate article comments
def sum_comment(x):
  d = []
  d.extend(x)
  dd = list(itertools.chain.from_iterable(d))
  return dd

In [ ]:
# combine duplicate article sentiment
from operator import itemgetter
def sum_sentiment(x):
   init_dict = list(x)

   result = {}
   for d in init_dict:
      for k in d.keys():
          result[k] = result.get(k, 0) + d[k]
   return result

In [ ]:
# remove duplicate article with same date, title, content and press
# combine duplicate article comments
# sum duplicate article comments count
# combine duplicate article sentiment
df_dedup_fin = df_dedup.groupby(['Date', 'Title', 'Content', 'Press'], as_index=False).agg({'ID' : 'last', 'Content_len' : 'last', 'Comment' : lambda x : sum_comment(x) , 'Comment_cnt' : 'sum', 'Sentiment' : lambda x : sum_sentiment(x), 'Link' : 'last'})

In [ ]:
for i in range(len(df_dedup_fin)):
  if df_dedup_fin['Comment_cnt'][i] == 0:
    df_dedup_fin['Comment'][i] = "NA"

In [ ]:
df_dedup_fin

In [ ]:
# add new column Title_clean
df_dedup_fin['Title_clean'] = "NA"
df_dedup_fin.head()

In [ ]:
len(df_dedup_fin)

In [ ]:
import re

In [ ]:
# remove special characters from text
def clean_text(readData):
  text = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', readData)
  text_rr = ' '.join(text.split())
  return text_rr

In [ ]:
df_dedup_fin = df_dedup_fin.reset_index(drop = True)

In [ ]:
# add the removed special character title to Title_clean column
df_dedup_fin['Title_clean']= df_dedup_fin['Title'].map(lambda x: clean_text(x))

In [ ]:
# rename column
df_dedup_fin.rename(columns={'Comment' : 'D_Comment', 'Comment_cnt' : 'D_Comment_cnt', 'Sentiment' : 'D_Sentiment'}, inplace=True)
# delete unnecessary columns
df_dedup_fin = df_dedup_fin[['ID', 'Date', 'Title', 'Content', 'Content_len', 'D_Comment', 'D_Comment_cnt', 'D_Sentiment', 'Link', 'Press', 'Title_clean']]

In [ ]:
df_dedup_fin

In [ ]:
len(df_dedup_fin)

In [ ]:
# save binary file using pickle
with open(link_d_dedup, 'wb') as f:
     pickle.dump(df_dedup_fin, f)

In [ ]:
# load binary file using pickle
with open(link_d_dedup, 'rb') as f:
  dd = pickle.load(f)

In [ ]:
dd